# RAG with chat history

<img src="/home/mauricio/Documents/Projects/RAG-Mastery/Diagrams/RAG-chat-history.png" width="40%">

We will copy part of the code we previously created in the Simple_rag folder. More specifically the load, split and retrieve documents. The part that will be diferent is that the chain is going to be a history retriever.

In [1]:
import sys
import os
import tqdm as notebook_tqdm
from typing import List, Union

# Path to the directory containing config.py
config_path = '/home/mauricio/Documents/Projects/RAG-Mastery'

# Add the directory to sys.path
if config_path not in sys.path:
    sys.path.append(config_path)

# Now you can import the API_KEY from config.py
from config import API_KEY

path_to_docs = "/home/mauricio/Documents/Projects/RAG-Mastery/data"

In [2]:
from langchain_mistralai.chat_models import ChatMistralAI
def get_llm_model(self):
        return ChatMistralAI(
            model_name="open-mixtral-8x22b", 
            mistral_api_key=self.API_KEY
        )
from langchain_unstructured import UnstructuredLoader
from langchain.schema.document import Document

def load_documents(folder_path):
    documents = []
    for file in os.listdir(folder_path):
        if file.endswith('.docx') or file.endswith('.pdf') or file.endswith('.txt'):
            loader = UnstructuredLoader(os.path.join(folder_path, file))
            documents.extend(loader.load())
        print("Document loaded lenght: ", len(documents))
    print("Documents loaded successfully ✅")
    print(documents[0].metadata.get("filename"))
    return documents
from langchain_text_splitters import RecursiveCharacterTextSplitter


def split_documents(documents, chunk_size= 1000, chunk_overlap= 200):
        try:
            text_splitter: RecursiveCharacterTextSplitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size,
                chunk_overlap=chunk_overlap,
                length_function=len,
                separators=["\n\n", "\n", " ", ""]
            )
            splits: List[Document] = text_splitter.split_documents(documents)
            print("Split document successfully ✅")
            print("Documents split: ", len(splits))
            return splits
        except Exception as e:
            print(f"Error splitting documents: {e}")
            raise
from langchain_mistralai import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS

def embed_documents(splits):
    try:
        embeddings = MistralAIEmbeddings(
            model="mistral-embed",
            mistral_api_key=API_KEY
            )
        vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
        retriever = vectorstore.as_retriever(
            search_type="mmr",
            search_kwargs={"k": 6},
        )

        print("Retriever succesfuly created ✅")
        return retriever
    except Exception as e:
        print(f"Error embeding/retrieving documents {e}")
        raise

Now let's beggin the construction of the history chat. We will stat by adding a sub-chain that takes the lastest user question and reformulates it in the context of the chat history, basically qe want to do this

(query, conversation history) -> LLM -> rephrased query -> retriever



In [6]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
    )

